# Project - Airline AI Assistant

We'll now bring together what we've learned to make an AI Customer Support assistant for an Airline

In [1]:
import os
import json
import ollama
import gradio as gr

In [2]:
# Initialization

MODEL = "llama3.2"

# Verify Ollama is available
try:
    ollama.list()
    print("Ollama is available and ready to use")
except Exception as e:
    print(f"Ollama may not be running: {e}")


Ollama is available and ready to use


In [3]:
system_message = """
You are a helpful assistant for an Airline called FlightAI.
Give short, courteous answers, no more than 1 sentence.
Always be accurate. If you don't know the answer, say so.
"""

In [4]:
def chat(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = ollama.chat(model=MODEL, messages=messages)
    return response['message']['content']

gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7877
* To create a public link, set `share=True` in `launch()`.


## Tools

Tools are an incredibly powerful feature provided by the frontier LLMs.

With tools, you can write a function, and have the LLM call that function as part of its response.

Sounds almost spooky.. we're giving it the power to run code on our machine?

Well, kinda.

In [5]:
# Let's start by making a useful function
# NOTE: 이 함수는 나중에 데이터베이스 버전으로 대체됩니다 (Cell 26 참조)

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    # 안전성 체크: 딕셔너리나 None이 전달되는 경우 처리
    if isinstance(destination_city, dict):
        destination_city = destination_city.get('destination_city', '')
    if not destination_city or not isinstance(destination_city, str):
        return "Invalid city name provided"
    
    print(f"Tool called for city {destination_city}")
    price = ticket_prices.get(destination_city.lower(), "Unknown ticket price")
    return f"The price of a ticket to {destination_city} is {price}"


In [6]:
get_ticket_price("London")

Tool called for city London


'The price of a ticket to London is $799'

In [7]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [8]:
# set_ticket_price tool 정의 추가
set_price_function = {
    "name": "set_ticket_price",
    "description": "Set or update the price of a return ticket for a destination city. Use this when a customer or admin wants to change ticket prices.",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The destination city name (e.g., 'London', 'Paris', 'Tokyo')",
            },
            "price": {
                "type": "number",
                "description": "The ticket price in dollars (e.g., 799, 899.50)",
            },
        },
        "required": ["city", "price"],
        "additionalProperties": False
    }
}

# And this is included in a list of tools:

tools = [
    {"type": "function", "function": price_function},
    {"type": "function", "function": set_price_function}
]

In [9]:
tools

[{'type': 'function',
  'function': {'name': 'get_ticket_price',
   'description': 'Get the price of a return ticket to the destination city.',
   'parameters': {'type': 'object',
    'properties': {'destination_city': {'type': 'string',
      'description': 'The city that the customer wants to travel to'}},
    'required': ['destination_city'],
    'additionalProperties': False}}},
 {'type': 'function',
  'function': {'name': 'set_ticket_price',
   'description': 'Set or update the price of a return ticket for a destination city. Use this when a customer or admin wants to change ticket prices.',
   'parameters': {'type': 'object',
    'properties': {'city': {'type': 'string',
      'description': "The destination city name (e.g., 'London', 'Paris', 'Tokyo')"},
     'price': {'type': 'number',
      'description': 'The ticket price in dollars (e.g., 799, 899.50)'}},
    'required': ['city', 'price'],
    'additionalProperties': False}}}]

## Getting Ollama to use our Tool

There's some fiddly stuff to allow Ollama "to call our tool"

What we actually do is give the LLM the opportunity to inform us that it wants us to run the tool.

Here's how the new chat function looks:

In [10]:
# We have to write that function handle_tool_call:

def handle_tool_call(message):
    """
    단일 tool 호출을 처리합니다 (하나의 tool_call만 있을 때 사용)
    """
    tool_call = message['tool_calls'][0]
    tool_name = tool_call['function']['name']
    
    # Arguments 파싱
    arguments = tool_call['function'].get('arguments', {})
    if isinstance(arguments, str):
        try:
            arguments = json.loads(arguments)
        except json.JSONDecodeError:
            arguments = {}
    elif not isinstance(arguments, dict):
        arguments = {}
    
    tool_call_id = tool_call.get('id')
    
    if tool_name == "get_ticket_price":
        city = arguments.get('destination_city') if isinstance(arguments, dict) else None
        if not city:
            price_details = "Error: destination_city parameter is required"
        else:
            price_details = get_ticket_price(city)
        
        response = {
            "role": "tool",
            "content": price_details
        }
        if tool_call_id:
            response["tool_call_id"] = tool_call_id
            
    elif tool_name == "set_ticket_price":
        city = arguments.get('city') or arguments.get('destination_city')
        price = arguments.get('price')
        
        if not city or price is None:
            result = "Error: Both city and price parameters are required"
        else:
            try:
                price = float(price)
                set_ticket_price(city, price)
                result = f"Successfully set ticket price for {city} to ${price}"
            except ValueError:
                result = f"Error: Invalid price value: {price}"
            except Exception as e:
                result = f"Error setting ticket price: {str(e)}"
        
        response = {
            "role": "tool",
            "content": result
        }
        if tool_call_id:
            response["tool_call_id"] = tool_call_id
    else:
        # 알 수 없는 tool
        response = {
            "role": "tool",
            "content": f"Unknown tool: {tool_name}"
        }
        if tool_call_id:
            response["tool_call_id"] = tool_call_id
    
    return response

In [11]:
def chat(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    # ai 호출 시 JSON 추가
    response = ollama.chat(model=MODEL, messages=messages, tools=tools)

    # 도구 호출 여부 확인
    if response['message'].get('tool_calls'):
        assistant_message = response['message']
        tool_response = handle_tool_call(assistant_message)
        messages.append(assistant_message)
        messages.append(tool_response)
        response = ollama.chat(model=MODEL, messages=messages)
    
    return response['message']['content']

In [12]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7878
* To create a public link, set `share=True` in `launch()`.


## Let's make a couple of improvements

Handling multiple tool calls in 1 response

Handling multiple tool calls 1 after another

In [13]:
def chat(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = ollama.chat(model=MODEL, messages=messages, tools=tools)

    if response['message'].get('tool_calls'):
        assistant_message = response['message']
        responses = handle_tool_calls(assistant_message)
        messages.append(assistant_message)
        messages.extend(responses)
        response = ollama.chat(model=MODEL, messages=messages)
    
    return response['message']['content']

In [14]:
def handle_tool_calls(message):
    responses = []
    for tool_call in message['tool_calls']:
        tool_name = tool_call['function']['name']
        
        # Arguments 파싱: dict, JSON string, 또는 다른 형식 처리
        arguments = tool_call['function'].get('arguments', {})
        if isinstance(arguments, str):
            try:
                arguments = json.loads(arguments)
            except json.JSONDecodeError:
                arguments = {}
        elif not isinstance(arguments, dict):
            arguments = {}
        
        tool_call_id = tool_call.get('id')
        
        if tool_name == "get_ticket_price":
            city = arguments.get('destination_city') if isinstance(arguments, dict) else None
            if not city:
                price_details = "Error: destination_city parameter is required"
            else:
                price_details = get_ticket_price(city)
            
            response = {
                "role": "tool",
                "content": price_details
            }
            if tool_call_id:
                response["tool_call_id"] = tool_call_id
            responses.append(response)
            
        elif tool_name == "set_ticket_price":
            city = arguments.get('city') or arguments.get('destination_city')
            price = arguments.get('price')
            
            if not city or price is None:
                result = "Error: Both city and price parameters are required"
            else:
                try:
                    price = float(price)  # 가격을 숫자로 변환
                    set_ticket_price(city, price)
                    result = f"Successfully set ticket price for {city} to ${price}"
                except ValueError:
                    result = f"Error: Invalid price value: {price}"
                except Exception as e:
                    result = f"Error setting ticket price: {str(e)}"
            
            response = {
                "role": "tool",
                "content": result
            }
            if tool_call_id:
                response["tool_call_id"] = tool_call_id
            responses.append(response)
        else:
            # 알 수 없는 tool
            response = {
                "role": "tool",
                "content": f"Unknown tool: {tool_name}"
            }
            if tool_call_id:
                response["tool_call_id"] = tool_call_id
            responses.append(response)
    
    return responses

In [15]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7879
* To create a public link, set `share=True` in `launch()`.


In [16]:
def chat(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = ollama.chat(model=MODEL, messages=messages, tools=tools)

    while response['message'].get('tool_calls'):
        assistant_message = response['message']
        responses = handle_tool_calls(assistant_message)
        messages.append(assistant_message)
        messages.extend(responses)
        response = ollama.chat(model=MODEL, messages=messages, tools=tools)
    
    return response['message']['content']

In [17]:
import sqlite3


In [18]:
DB = "prices.db"

with sqlite3.connect(DB) as conn:
    cursor = conn.cursor()
    cursor.execute('CREATE TABLE IF NOT EXISTS prices (city TEXT PRIMARY KEY, price REAL)')
    conn.commit()

In [19]:
def get_ticket_price(city):
    # 안전성 체크: city가 딕셔너리나 None인 경우 처리
    if isinstance(city, dict):
        city = city.get('destination_city', '')
    if not city or not isinstance(city, str):
        return "Invalid city name provided"
    
    print(f"DATABASE TOOL CALLED: Getting price for {city}", flush=True)
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT price FROM prices WHERE city = ?', (city.lower(),))
        result = cursor.fetchone()
        return f"Ticket price to {city} is ${result[0]}" if result else "No price data available for this city"

In [20]:
get_ticket_price("London")

DATABASE TOOL CALLED: Getting price for London


'Ticket price to London is $799.0'

In [21]:
def set_ticket_price(city, price):
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('INSERT INTO prices (city, price) VALUES (?, ?) ON CONFLICT(city) DO UPDATE SET price = ?', (city.lower(), price, price))
        conn.commit()

In [22]:
ticket_prices = {"london":799, "paris": 899, "tokyo": 1420, "sydney": 2999}
for city, price in ticket_prices.items():
    set_ticket_price(city, price)

In [23]:
get_ticket_price("Tokyo")

DATABASE TOOL CALLED: Getting price for Tokyo


'Ticket price to Tokyo is $1420.0'

### 중요: 커널 재시작 필요

**에러가 발생하는 경우:**

에러 메시지에서 `'dict' object has no attribute 'lower'` 같은 문제가 발생하면, 노트북 커널을 재시작해야 합니다:

1. **커널 재시작**: `Kernel` → `Restart Kernel` (또는 `Restart & Clear Output`)
2. **셀 순서대로 다시 실행**: 
   - Cell 1: imports
   - Cell 2: 초기화
   - Cell 3: system_message
   - Cell 9: tools 정의 (set_ticket_price 포함)
   - Cell 14: handle_tool_call
   - Cell 21: handle_tool_calls
   - Cell 25: 데이터베이스 설정
   - Cell 26: get_ticket_price (데이터베이스 버전)
   - Cell 28: set_ticket_price
   - Cell 23 또는 Cell 31: chat 함수
3. **Gradio 인터페이스 실행**

이렇게 하면 최신 버전의 함수들이 메모리에 로드됩니다.

In [24]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7880
* To create a public link, set `share=True` in `launch()`.


## Exercise

Add a tool to set the price of a ticket!

In [25]:
def set_ticket_price(city, price):
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('INSERT INTO prices (city, price) VALUES (?, ?) ON CONFLICT(city) DO UPDATE SET price = ?', (city.lower(), price, price))
        conn.commit()

In [26]:
set_ticket_price("Seoul", "$3000")

DATABASE TOOL CALLED: Getting price for your desired destination city
DATABASE TOOL CALLED: Getting price for London
DATABASE TOOL CALLED: Getting price for London
DATABASE TOOL CALLED: Getting price for Toko
DATABASE TOOL CALLED: Getting price for Tokyo
DATABASE TOOL CALLED: Getting price for Tokyo
DATABASE TOOL CALLED: Getting price for London


<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business Applications</h2>
            <span style="color:#181;">Hopefully this hardly needs to be stated! You now have the ability to give actions to your LLMs. This Airline Assistant can now do more than answer questions - it could interact with booking APIs to make bookings!</span>
        </td>
    </tr>
</table>